In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
osm=pd.read_csv('OSM.csv')
osm.head(1)

,FID,OBJECTID,Join_Count,osm_id,BUFF_DIST,Shape_Leng
0,0,1,3,26130796,25.0,0.001527


In [ ]:
osm_fid=osm['FID'].values.tolist()
sv=pd.read_csv('SV.csv')
sv['0517ID']=sv['nid'].apply(lambda x: x[0:23])
# df['age']=df['age'].apply(lambda x: x+3)
sv.head(1)

,FID,descriptio,tags,lat_wgs,lon_wgs,pov_exp,heading,orgid,nid,scene,...,F13,F14,F15,F16,F17,F18,F19,F20,F21,0517ID
0,0,Ｇ１０７,GENERAL,30.612083,114.164546,2.0,107.0,10141003150599999000000.000000000000000,10141003150629162530900_right_28.jpg,a road with cars driving on it ; a group of ca...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10141003150629162530900


*   target_fid是街景的ID
*   join_fid是OSM重建的ID(0-5000+)
*   OSM FID是从0开始的 org_fid

In [ ]:
sv.shape

(18907, 23)

In [ ]:
sv_osm=pd.read_csv('OSMSV.csv')
print(sv_osm.columns)
sv_osm.head(1)

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'JOIN_FID', 'descriptio',
       'tags', 'lat_wgs', 'lon_wgs', 'pov_exp', 'heading', 'orgid', 'nid',
       'scene', 'label', 'multi_labe', 'BUFF_DIST', 'ORIG_FID', 'OBJECTID_1',
       'Join_Count_1', 'osm_id', 'Shape_Length', 'Shape_Area'],
      dtype='object')


,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,descriptio,tags,lat_wgs,lon_wgs,pov_exp,heading,...,scene,label,multi_labe,BUFF_DIST,ORIG_FID,OBJECTID_1,Join_Count_1,osm_id,Shape_Length,Shape_Area
0,1,29,1,111,Ｇ１０７,GENERAL,30.612083,114.164546,2.0,107.0,...,a road with cars driving on it ; a group of ca...,9.0,[],30.0,0,1,1,58871382,0.001833,2.648248e-07


In [ ]:
sv_fid=list(set(sv_osm['ORIG_FID'].values.tolist()))
print(sv_fid[0])
print(len(sv_fid))

0
13889


In [ ]:
sv_osm.shape

(28693, 22)

In [ ]:
bd=pd.read_csv('BDSV.csv')
bd.head(1)

,FID,descriptio,tags,lat_wgs,lon_wgs,pov_exp,heading,orgid,nid,scene,...,F13,F14,F15,F16,F17,F18,F19,F20,F21,bdscene
0,0,Ｇ１０７,GENERAL,30.612083,114.164546,2.0,107.0,10141003150599999000000.000000000000000,10141003150629162530900_right_28.jpg,a road with cars driving on it ; a group of ca...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,a group of cars driving down a street;a parkin...


In [ ]:
sv=bd
sv.shape

(18907, 23)

In [ ]:
result=pd.merge(sv_osm,sv[['nid','bdscene']],left_on='nid',right_on='nid')
result[result['bdscene']==None].shape

(0, 23)

In [ ]:
result.tail()['bdscene'].values.tolist()

['a truck driving down a road next to a building;a person riding a motorcycle on a city street;the side of a building with graffiti on it;a person riding a motorcycle on a city street',
 'a truck driving down a road next to a building;a person riding a motorcycle on a city street;the side of a building with graffiti on it;a person riding a motorcycle on a city street',
 'a truck driving down a road next to a bridge;a group of cars parked on the side of the road;a white car parked on the side of a road;a city street with a blue fence and trees',
 nan,
 'a car driving down a street next to a building;a view of a city street with tall buildings;a view of a street corner with a building in the background;a large red building on the side of the street']

In [ ]:
result.shape

(28693, 23)

In [ ]:
sv_osm=result

In [ ]:
osm_sv_scene={}
osm_sv_id={}
osm_sv_imgpath={}

osm_sv_label={}
osm_sv_wuhan={}

for i in osm_fid:
  osm_sv_id[i]=sv_osm[sv_osm['JOIN_FID']==i]['ORIG_FID'].values.tolist()
  osm_sv_scene[i]=sv_osm[sv_osm['JOIN_FID']==i]['bdscene'].values.tolist()
  osm_sv_imgpath[i]=sv_osm[sv_osm['JOIN_FID']==i]['nid'].values.tolist()
  #来计算标签
  osm_sv_label[i]=sv_osm[sv_osm['JOIN_FID']==i]['multi_labe'].values.tolist()
  # osm_sv_wuhan[i]=sv_osm[sv_osm['JOIN_FID']==i]['type'].values.tolist()

In [ ]:
osm_sv_scene[0]

['a tree that is next to some water;a view of a city skyline from a distance;a person riding a bike down a street;a car driving down a street near trees',
 nan,
 'a couple of people that are standing in the street;a white car driving down a road next to trees;a road with a blue street sign next to it']

In [ ]:
import torch
print(torch.__version__)  #注意是双下划线

1.12.0+cu113


In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 3.5 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 407 kB 4.2 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=529ffd623cc3d803e85d09e00c56dcbb22f01c0a4d9d7b288723e8d7546e7982
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [ ]:
with open('OsmRoadJoinLarge1_buffer_forgeoda_k5.gwt','r+') as f:
  edges=f.readlines()
edge_index=[]
for edge in edges[1:]:
  edge_index.append(list(map(int,edge.split(' ')[0:2])))
edge_index = torch.tensor(edge_index, dtype=torch.long) 
len(edge_index)

27290

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 4.4 MB 27.5 MB/s 
     |████████████████████████████████| 1.2 MB 48.8 MB/s 
     |████████████████████████████████| 101 kB 10.0 MB/s 
     |████████████████████████████████| 596 kB 42.7 MB/s 
     |████████████████████████████████| 6.6 MB 30.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=39fa2ef85e524cb3691b4df58cc90c4fcb7406d2a6c7a43012d9e979046674aa
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
scene_list=[]
for fid in osm_fid:
  # print(fid)
  osm_sv_scene[0]
  if len(osm_sv_scene[fid])>0:
    scene_list.append( ','.join(str(v) for v in osm_sv_scene[fid]))
  else:
    scene_list.append('')

In [ ]:
%time
embeddings = model.encode(scene_list)
# embeddings=pd.read_csv('/content/drive/MyDrive/路段功能识别/百度osmembd0719.csv').values[:,1:].T
# embeddings.shape

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


In [ ]:
osm_sv_embd={}
for index, value in enumerate(osm_fid):
	osm_sv_embd[value]=embeddings[index]
pd.DataFrame(osm_sv_embd).to_csv('/content/drive/MyDrive/路段功能识别/百度osmembd0719.csv')
# pd.DataFrame(osm_sv_embd).to_csv('/content/drive/MyDrive/路段功能识别/osmembd0520DOC分割图嵌入.csv')

In [ ]:
labels=[101,201,202,301,402,501,502,503,504,505]
slabels=["['101']","['201']","['202']","['301']","['402']","['501']","['502']","['503']","['504']","['505']"]
types_label=['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9']
#{' ', '0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', nan}
#计算出标签以及标签个数 old
osm_fid_label={}
for fid in osm_fid:
  tmp=osm_sv_label[fid]
  max_label=max(tmp,key=tmp.count)
  max_label_num=tmp.count(max(tmp,key=tmp.count))
  if max_label in slabels:
    res_label=slabels.index(max_label)
  else:
    res_label=-1
  osm_fid_label[fid]=[res_label,max_label_num]

In [ ]:
#提取大于2数量的值
label_mask=[]
label_y=[]
for fid in osm_fid:
  if osm_fid_label[fid][1]>2 and osm_fid_label[fid][0]!=-1:
    label_y.append(osm_fid_label[fid][0])
    label_mask.append(True)
  else:
    label_y.append(-1)
    label_mask.append(False)

In [ ]:
from collections import Counter
result = Counter(label_y)
result

Counter({-1: 4930,
         0: 320,
         1: 15,
         2: 29,
         3: 37,
         4: 6,
         5: 6,
         6: 65,
         7: 12,
         8: 28,
         9: 10})

In [ ]:
#提取大于2数量的值
label_mask=[]
label_y=[]
balanced_label=0
for fid in osm_fid:
  if osm_fid_label[fid][0]==0:
    balanced_label=balanced_label+1
  if osm_fid_label[fid][1]>2 and osm_fid_label[fid][0]!=-1:
    if balanced_label>200 and osm_fid_label[fid][0]==0:
      label_y.append(-1)
      label_mask.append(False)
    else:
      label_y.append(osm_fid_label[fid][0])
      label_mask.append(True) 
  else:
    label_y.append(-1)
    label_mask.append(False)
from collections import Counter
result = Counter(label_y)
result

Counter({-1: 5163,
         0: 87,
         1: 15,
         2: 29,
         3: 37,
         4: 6,
         5: 6,
         6: 65,
         7: 12,
         8: 28,
         9: 10})

In [ ]:
import numpy as np
pd.DataFrame({'Origin':np.array(label_y)}).to_csv('Origin.csv')

In [ ]:
# Install required packages.
import os
# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])
    
    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.legend()
    plt.show()
# fig, ax = plt.subplots()
# ax.plot(a, c, 'k--', label='Model length')
# ax.plot(a, d, 'k:', label='Data length')
# ax.plot(a, c + d, 'k', label='Total message length')
# legend = ax.legend(loc='upper center', shadow=True, fontsize='x-large')
# legend.get_frame().set_facecolor('C0')

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html 
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
!pip install torch-geometric

In [ ]:
# !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cpu.html

In [ ]:
# !pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
# !pip install torch-geometric

In [ ]:
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

In [ ]:
!pip uninstall torch-sparse
!pip uninstall torch-sparse
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu102.html

Found existing installation: torch-sparse 0.6.14
Uninstalling torch-sparse-0.6.14:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/test/*
    /usr/local/lib/python3.7/dist-packages/torch_sparse-0.6.14.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch_sparse/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/test/test_basis.py
    /usr/local/lib/python3.7/dist-packages/test/test_broadcasting.py
    /usr/local/lib/python3.7/dist-packages/test/test_conv.py
    /usr/local/lib/python3.7/dist-packages/test/test_fps.py
    /usr/local/lib/python3.7/dist-packages/test/test_gather.py
    /usr/local/lib/python3.7/dist-packages/test/test_graclus.py
    /usr/local/lib/python3.7/dist-packages/test/test_grid.py
    /usr/local/lib/python3.7/dist-packages/test/test_knn.py
    /usr/local/lib/python3.7/dist-packages/test/test_multi_gpu.py
    /usr/local/lib/python3.7/dist-packages/test/test_nearest.py
    /usr/local/lib/python3.7/dist-packages

In [ ]:
# # Install required packages.
# import os
# import torch
# os.environ['TORCH'] = torch.__version__
# print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# # Helper function for visualization.
# %matplotlib inline
# import networkx as nx
# import matplotlib.pyplot as plt

# import torch
# from torch_geometric.data import Data
# import numpy as np

In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html

In [ ]:
 !pip install torch=='1.9.0' !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.9.0+cu102.html

ERROR: Invalid requirement: '!pip'


In [ ]:
# edge_index = edge_index
x = torch.tensor(embeddings, dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous(),
y=torch.from_numpy(np.array(label_y)),
train_mask=torch.from_numpy(np.array(label_mask)),
test_mask=torch.from_numpy(np.array(label_mask)))